In [98]:
import pandas as pd

import spacy
import collections

from bs4 import BeautifulSoup as BS

nlp = spacy.load('de_core_news_sm')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [99]:
df_tag_category = pd.read_parquet('data/tag_category.parquet')
df_tag = pd.read_parquet('data/tag.parquet')

df_event = pd.read_parquet('data/event.parquet')
df_event_tags = pd.read_parquet('data/event_tags.parquet')
df_event_material_list = pd.read_parquet('data/event_material_list.parquet')

df_experiment = pd.read_parquet('data/experiment.parquet')
df_experiment_item = pd.read_parquet('data/experiment_item.parquet')

df_feature_vector = pd.read_parquet('data/feature_vector.parquet')

In [100]:
df_feature_vector = df_feature_vector[[
  'id',
  'title',
  'description',

  'p_tag',
  'img_tag',
  'b_tag',
  'li_tag',
  'header_img',
  'text_len',
  'span_tag',
  'a_tag',
  'u_tag',

  'costsRating',
  'executionTimeRating',
  'isPrepairationNeeded',

  'total_words',

  'nomen_quote',
  'adjektive_quote',
  'numerisch_quote',
  'satzzeichen_quote',
  'konjunktion_quote',

  'count_question_mark'
]]

In [101]:
def percentage_change(col1, col2):
    return col1 / col2 * 100

In [102]:
def calc_label_score(unklar, doof, gut, sehr_gut, count):
    score = (doof * 1 + gut * 3 + sehr_gut * 5) / count
    return score

,id,title,description,costsRating,executionTimeRating,isPrepairationNeeded,imageLink,created_at,createdBy,event_id_id,Bewertungen,doof,gut,sehr_gut,unklar,doof_rate,gut_rate,sehr_gut_rate,unklar_rate,calc_label_score


In [103]:
df_label_temp = df_experiment_item.groupby(["event_id_id"]).size().reset_index(name='Bewertungen')
df_1 = df_experiment_item[df_experiment_item['score'] == 1].groupby(["event_id_id"]).size().reset_index(name='doof')
df_2 = df_experiment_item[df_experiment_item['score'] == 2].groupby(["event_id_id"]).size().reset_index(name='gut')
df_3 = df_experiment_item[df_experiment_item['score'] == 3].groupby(["event_id_id"]).size().reset_index(name='sehr_gut')
df_4 = df_experiment_item[df_experiment_item['score'] == 4].groupby(["event_id_id"]).size().reset_index(name='unklar')

df_label_temp_2 = df_label_temp.join(
    df_1.set_index('event_id_id'), on='event_id_id').join(
    df_2.set_index('event_id_id'), on='event_id_id').join(
    df_3.set_index('event_id_id'), on='event_id_id').join(
    df_4.set_index('event_id_id'), on='event_id_id')

df_label_temp_2 = df_label_temp_2.fillna(0)

df_label = pd.merge(df_event, df_label_temp_2, left_on=['id'], right_on= ['event_id_id'], how = 'left')

In [104]:
df_label['doof_rate'] = percentage_change(df_label['doof'], df_label['Bewertungen'])
df_label['gut_rate'] = percentage_change(df_label['gut'], df_label['Bewertungen'])
df_label['sehr_gut_rate'] = percentage_change(df_label['sehr_gut'], df_label['Bewertungen'])
df_label['unklar_rate'] = percentage_change(df_label['unklar'], df_label['Bewertungen'])

df_label['calc_label_score'] = calc_label_score(df_label['unklar'], df_label['doof'], df_label['gut'], df_label['sehr_gut'], df_label['Bewertungen'])

# df = pd.merge(df_label, event_tags_features, left_on=['id'], right_on= ['event_id'], how = 'left')
# df_raw = df

df_label = df_label[df_label['Bewertungen'] > 1]
df_label = df_label[df_label['unklar'] == 0]

df_label = df_label.round(2)

df_label.to_parquet('data/label_v1.parquet', coerce_timestamps="us")

In [105]:
df_label.nlargest(5, 'unklar_rate')

,id,title,description,costsRating,executionTimeRating,isPrepairationNeeded,imageLink,created_at,createdBy,event_id_id,Bewertungen,doof,gut,sehr_gut,unklar,doof_rate,gut_rate,sehr_gut_rate,unklar_rate,calc_label_score
1,2,Bist du Bündisch oder Scoutistisch?,<p>Ihr k&ouml;nnt verschiedene Situationen aus...,0,1,0,https://api.xrdcx.de/media/images/3a63be00-7f8...,2021-03-07 20:00:27.660006,Robert,2.0,2.0,0.0,2.0,0.0,0.0,0.0,100.0,0.0,0.0,3.0
2,3,Behelfstrage aus Naturmaterialien bauen,<p>Ihr k&ouml;nnt in den Wald gehen und dort a...,0,3,0,None,2021-03-07 20:00:27.660006,Robert,3.0,2.0,0.0,2.0,0.0,0.0,0.0,100.0,0.0,0.0,3.0
3,4,Gepäckteile besprechen und wiegen,<p>Ihr kommt mit einem gepackten Rucksack zum ...,0,3,1,None,2021-03-07 20:00:27.660006,Robert,4.0,3.0,0.0,3.0,0.0,0.0,0.0,100.0,0.0,0.0,3.0
4,5,Sippen-Holzbrettchen erstellen,<p>Ihr k&ouml;nnt blanke Holzbrettchen g&uuml;...,2,3,1,None,2021-03-07 20:00:27.660006,Robert,5.0,2.0,0.0,2.0,0.0,0.0,0.0,100.0,0.0,0.0,3.0
5,6,Blattbuch erstellen,<p>Ihr sollt ein Blattbuch erstellen.</p>\n<p>...,1,3,1,https://api.xrdcx.de/media/images/15f62406-86a...,2021-03-07 20:00:27.660006,Robert,6.0,2.0,0.0,1.0,1.0,0.0,0.0,50.0,50.0,0.0,4.0


In [106]:
df_label.nlargest(5, 'gut_rate')

,id,title,description,costsRating,executionTimeRating,isPrepairationNeeded,imageLink,created_at,createdBy,event_id_id,Bewertungen,doof,gut,sehr_gut,unklar,doof_rate,gut_rate,sehr_gut_rate,unklar_rate,calc_label_score
1,2,Bist du Bündisch oder Scoutistisch?,<p>Ihr k&ouml;nnt verschiedene Situationen aus...,0,1,0,https://api.xrdcx.de/media/images/3a63be00-7f8...,2021-03-07 20:00:27.660006,Robert,2.0,2.0,0.0,2.0,0.0,0.0,0.0,100.0,0.0,0.0,3.0
2,3,Behelfstrage aus Naturmaterialien bauen,<p>Ihr k&ouml;nnt in den Wald gehen und dort a...,0,3,0,None,2021-03-07 20:00:27.660006,Robert,3.0,2.0,0.0,2.0,0.0,0.0,0.0,100.0,0.0,0.0,3.0
3,4,Gepäckteile besprechen und wiegen,<p>Ihr kommt mit einem gepackten Rucksack zum ...,0,3,1,None,2021-03-07 20:00:27.660006,Robert,4.0,3.0,0.0,3.0,0.0,0.0,0.0,100.0,0.0,0.0,3.0
4,5,Sippen-Holzbrettchen erstellen,<p>Ihr k&ouml;nnt blanke Holzbrettchen g&uuml;...,2,3,1,None,2021-03-07 20:00:27.660006,Robert,5.0,2.0,0.0,2.0,0.0,0.0,0.0,100.0,0.0,0.0,3.0
12,38,Pflanzen/Tiere im Gewässer bestimmen,"<p>Ihr k&ouml;nnt einen naturnahen Fluss, Bach...",1,3,1,None,2021-03-07 20:00:27.660006,Madita,38.0,3.0,0.0,3.0,0.0,0.0,0.0,100.0,0.0,0.0,3.0


In [107]:
df_label.nlargest(5, 'sehr_gut_rate')

,id,title,description,costsRating,executionTimeRating,isPrepairationNeeded,imageLink,created_at,createdBy,event_id_id,Bewertungen,doof,gut,sehr_gut,unklar,doof_rate,gut_rate,sehr_gut_rate,unklar_rate,calc_label_score
24,59,Fensterbank-Kräuterbeet,"<p><img style=""margin-left: 20px; margin-right...",2,2,1,None,2021-03-07 20:00:27.660006,Dorothea,59.0,2.0,0.0,0.0,2.0,0.0,0.0,0.00,100.00,0.0,5.00
101,165,Digitale Karten?,<p>Am Heimabend k&ouml;nnt ihr verschiedene Ka...,1,3,1,https://api.xrdcx.de/media/images/44784fa8-800...,2021-03-07 20:00:27.660006,Robert,165.0,2.0,0.0,0.0,2.0,0.0,0.0,0.00,100.00,0.0,5.00
95,158,Pfeil und Bogen bauen,<p>Am Heimabend k&ouml;nnt ihr einfach selber ...,1,3,1,https://api.xrdcx.de/media/images/eb6911f4-869...,2021-03-07 20:00:27.660006,Robert,158.0,3.0,0.0,1.0,2.0,0.0,0.0,33.33,66.67,0.0,4.33
120,201,Lichtmalerei,"<p style=""text-align: left;"">Male mit einer Ta...",0,1,1,https://api.xrdcx.de/media/images/f4814142-7f8...,2021-03-07 20:00:27.660006,Leyan,201.0,3.0,0.0,1.0,2.0,0.0,0.0,33.33,66.67,0.0,4.33
96,159,[Nur für Rover] Feuerfestigkeit testen,"<table style=""border-collapse: collapse; width...",2,2,1,https://api.xrdcx.de/media/images/23ed2ccc-7f8...,2021-03-07 20:00:27.660006,Daniel,159.0,5.0,0.0,2.0,3.0,0.0,0.0,40.00,60.00,0.0,4.20


In [108]:
df_label.nlargest(5, 'doof_rate')

,id,title,description,costsRating,executionTimeRating,isPrepairationNeeded,imageLink,created_at,createdBy,event_id_id,Bewertungen,doof,gut,sehr_gut,unklar,doof_rate,gut_rate,sehr_gut_rate,unklar_rate,calc_label_score
14,44,Anschleichen üben,<p>Ihr k&ouml;nnt eure F&auml;higkeiten im Ans...,0,1,0,https://api.xrdcx.de/media/images/c9bcb2b2-86a...,2021-03-07 20:00:27.660006,Robert,44.0,2.0,2.0,0.0,0.0,0.0,100.00,0.00,0.00,0.0,1.00
30,66,Milchkarton-Portmonnaie,<p>Ihr habt noch kein Portmonnaie f&uuml;r eur...,0,2,1,None,2021-03-07 20:00:27.660006,Dorothea,66.0,4.0,3.0,1.0,0.0,0.0,75.00,25.00,0.00,0.0,1.50
33,70,Jonglierbälle selber machen,<p>Wenn man nicht gerade mit &Auml;pfeln jongl...,1,1,1,None,2021-03-07 20:00:27.660006,Dorothea,70.0,3.0,2.0,0.0,1.0,0.0,66.67,0.00,33.33,0.0,2.33
69,129,Blüten Tafel im Bilderrahmen,"<p style=""text-align: left;"">Erstellt eine <st...",2,0,1,https://api.xrdcx.de/media/images/4958bf0c-86a...,2021-03-07 20:00:27.660006,Alina,129.0,3.0,2.0,1.0,0.0,0.0,66.67,33.33,0.00,0.0,1.67
99,163,Handcreme selber machen,<p>F&uuml;r deine selbst gemachte Handcreme br...,3,2,1,None,2021-03-07 20:00:27.660006,Nadine,163.0,3.0,2.0,1.0,0.0,0.0,66.67,33.33,0.00,0.0,1.67
